# Preprocessing intracranial EEG using MNE-python


*WIRED 2024*  
[Maansi Desai, PhD](https://maansidesai.github.io/)  
Postdoctoral Researcher in the [Hamilton Lab](https://slhs.utexas.edu/research/hamilton-lab)
<br>
Department of Speech, Language, and Hearing Sciences 
<br>
The University of Texas at Austin 

## Part 3: Plotting electrodes on brain using MNE brain.viz function


In [1]:
%matplotlib notebook

import mne
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import os
from mne_bids import read_raw_bids, print_dir_tree
from mne_bids.path import get_bids_path_from_fname
from bids import BIDSLayout
from ecog_preproc_utils import transformData
import bids 
import re  # regex for comparing channel names

## Download BIDS iEEG dataset

Here we will download an example iEEG dataset from [this dataset from the Hamilton lab](https://openneuro.org/datasets/ds004993/versions/1.1.1). If you have ran [`01_ieeg_preprocessing_MNE`](01_ieeg_preprocessing_MNE) and [`02_ieeg_preprocessing_MNE_epochs.ipynb`](02_ieeg_preprocessing_MNE_epochs.ipynb), then you should already have downloaded the necessary datasets locally. 

For this notebook we will use data from: 
 - `sub-W1`, `iemu`, `B3`, `movietrailers`
 - `sub-W2`, `iemu`, `B8`, `timit5`
 - `sub-W3`, `iemu`, `B8`, `timit4`

In [2]:
# This is the example participant's data that we will load for the tutorial,
# but there are more options.
subj = 'W2'
sess = 'iemu'
task = 'timit5'
acq = 'B8'
run = '01'

In [3]:
import mne_bids
mne_bids.__version__

'0.10'

In [4]:
# Change the data directory below to where your data are located. 
parent_dir = 'data/ds004993'  # This is on your local machine

ieeg_dir = f'{parent_dir}/sub-{subj}/ses-{sess}/ieeg/'
channel_path = f'{ieeg_dir}/sub-{subj}_ses-{sess}_task-{task}_acq-{acq}_run-{run}_channels.tsv'
raw_path = f'{ieeg_dir}/sub-{subj}_ses-{sess}_task-{task}_acq-{acq}_run-{run}_ieeg.edf'

bids_path = get_bids_path_from_fname(raw_path)
base_name = os.path.basename(raw_path).split('.')[0]

## BIDS layout

We can use `pybids` to show a little bit about the files in this BIDS dataset. We won't get as much into this, but if you'd like to try this tutorial on your own you may wish to delve into this more.

In [5]:
layout = BIDSLayout(parent_dir)

In [6]:
layout.get_tasks()

['movietrailers', 'timit5', 'timit4']

In [7]:
all_files = layout.get()
print("There are {} files in the layout.".format(len(all_files)))
print("\nThe first 10 files are:")
all_files[:10]

There are 30 files in the layout.

The first 10 files are:


[<BIDSFile filename='/Users/maansidesai/Desktop/git/WIRED-2024-Paris/notebooks/data/ds004993/CHANGES'>,
 <BIDSJSONFile filename='/Users/maansidesai/Desktop/git/WIRED-2024-Paris/notebooks/data/ds004993/dataset_description.json'>,
 <BIDSJSONFile filename='/Users/maansidesai/Desktop/git/WIRED-2024-Paris/notebooks/data/ds004993/participants.json'>,
 <BIDSDataFile filename='/Users/maansidesai/Desktop/git/WIRED-2024-Paris/notebooks/data/ds004993/participants.tsv'>,
 <BIDSFile filename='/Users/maansidesai/Desktop/git/WIRED-2024-Paris/notebooks/data/ds004993/README'>,
 <BIDSDataFile filename='/Users/maansidesai/Desktop/git/WIRED-2024-Paris/notebooks/data/ds004993/sub-W1/ses-iemu/ieeg/sub-W1_ses-iemu_task-movietrailers_acq-B3_run-01_channels.tsv'>,
 <BIDSJSONFile filename='/Users/maansidesai/Desktop/git/WIRED-2024-Paris/notebooks/data/ds004993/sub-W1/ses-iemu/ieeg/sub-W1_ses-iemu_task-movietrailers_acq-B3_run-01_coordsystem.json'>,
 <BIDSDataFile filename='/Users/maansidesai/Desktop/git/WIRED-2

In [8]:
print_dir_tree(parent_dir, max_depth=3)

|ds004993/
|--- .DS_Store
|--- .bidsignore
|--- CHANGES
|--- README
|--- dataset_description.json
|--- participants.json
|--- participants.tsv
|--- freesurfer/
|------ sub-W1/
|--------- W1_lateral_lh.png
|--------- W1_lateral_rh.png
|--------- W1_ventral.png
|--------- bem/
|--------- surf/
|------ sub-W2/
|--------- W2_lateral_rh.png
|--------- W2_ventral_rh.png
|--------- bem/
|--------- surf/
|------ sub-W3/
|--------- W3_frontal.png
|--------- W3_lateral_lh.png
|--------- W3_lateral_rh.png
|--------- bem/
|--------- surf/
|--- sub-W1/
|------ .DS_Store
|------ ses-iemu/
|--------- .DS_Store
|--------- sub-W1_ses-iemu_scans.tsv
|--------- ieeg/
|------ ses-mri/
|--------- anat/
|--- sub-W2/
|------ .DS_Store
|------ ses-iemu/
|--------- .DS_Store
|--------- sub-W2_ses-iemu_scans.tsv
|--------- ieeg/
|------ ses-mri/
|--------- anat/
|--- sub-W3/
|------ .DS_Store
|------ ses-iemu/
|--------- .DS_Store
|--------- sub-W3_ses-iemu_scans.tsv
|--------- ieeg/
|------ ses-mri/
|---------

## Let's load some iEEG data!

First, we will choose the relevant subject, session, task, acquisition, and run. Note that if you wish to change these variables, you may need to download the data yourself.

To show the capabilities of BIDS and contrast to when we don't use BIDS, we'll load the data in two ways. The data structure using BIDS will be called `raw`, the data structure without BIDS will be `raw_nobids`.

In [9]:
# Read data and extract parameters from BIDS files
raw = read_raw_bids(bids_path, verbose=True)

Extracting EDF parameters from /Users/maansidesai/Desktop/git/WIRED-2024-Paris/notebooks/data/ds004993/sub-W2/ses-iemu/ieeg/sub-W2_ses-iemu_task-timit5_acq-B8_run-01_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading events from data/ds004993/sub-W2/ses-iemu/ieeg/sub-W2_ses-iemu_task-timit5_acq-B8_run-01_events.tsv.
Reading channel info from data/ds004993/sub-W2/ses-iemu/ieeg/sub-W2_ses-iemu_task-timit5_acq-B8_run-01_channels.tsv.
Reading electrode coords from data/ds004993/sub-W2/ses-iemu/ieeg/sub-W2_ses-iemu_task-timit5_acq-B8_run-01_electrodes.tsv.


/var/folders/03/nrm13jr11b734k78bp62stg80000gq/T/ipykernel_10470/531307012.py:2: RuntimeWarning: Omitted 102 annotation(s) that were outside data range.
  raw = read_raw_bids(bids_path, verbose=True)


# Electrodes on the brain


In [10]:
brain_kwargs = dict(alpha=0.1, background="white", view='split',cortex="low_contrast")
brain = mne.viz.Brain(f'sub-{subj}', surf='pial', subjects_dir=f'{parent_dir}/freesurfer', alpha=0.5,\
                      background='white')
trans = mne.read_trans(f'{parent_dir}/freesurfer/sub-{subj}/bem/sub-{subj}_head-trans.fif')
brain.add_sensors(raw.info, trans=trans)

Using pyvistaqt 3d backend.

Channel types::	seeg: 106
